<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/length_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepsmiles

In [2]:
!wget file.weasoft.com/images.zip
!wget file.weasoft.com/summary.csv

--2024-02-12 04:21:38--  http://file.weasoft.com/images.zip
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265877516 (254M) [application/zip]
Saving to: ‘images.zip.3’

images.zip.3          0%[                    ]       0  --.-KB/s               ^C
--2024-02-12 04:21:39--  http://file.weasoft.com/summary.csv
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 326352 (319K) [text/csv]
Saving to: ‘summary.csv.2’

summary.csv.2       100%[===================>] 318.70K   399KB/s    in 0.8s    

2024-02-12 04:21:40 (399 KB/s) - ‘summary.csv.2’ saved [326352/326352]



In [3]:
!wget file.weasoft.com/256.pt -O 256.pt
!unzip images.zip

--2024-02-12 04:21:40--  http://file.weasoft.com/256.pt
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153353860 (146M) [application/vnd.snesdev-page-table]
Saving to: ‘256.pt’

256.pt               58%[==========>         ]  85.11M  17.7MB/s    eta 4s     ^C
Archive:  images.zip
replace images/10952_050583472155929066.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import torch, torchvision
eff = torchvision.models.efficientnet_v2_s()
mynet = eff.features
class ImageEncoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.eff = mynet.to(device)
  def forward(self, images):
    features = self.eff(images)
    return features

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [3]:
import pandas as pd
molecules_rows = pd.read_csv("summary.csv")

In [4]:
def get_length(row):
  return len(converter.encode(row["canonicalsmiles"]))

In [5]:
import deepsmiles
converter = deepsmiles.Converter(rings=True, branches=True)

molecules_rows["canonicalsmiles"] = molecules_rows.apply(get_length, axis=1, )

In [6]:
import numpy as np
print(np.mean(molecules_rows["canonicalsmiles"]), np.std(molecules_rows["canonicalsmiles"]))

9.35820895522388 3.348875919840412


In [7]:
molecules_rows["canonicalsmiles"] = (molecules_rows["canonicalsmiles"])/np.std(molecules_rows["canonicalsmiles"])

In [8]:
import random, deepsmiles, os
from PIL import Image
import numpy as np

cids = list(molecules_rows["cid"])
train_cids = [i for i in cids if not i in [286, 6587, 6562, 11203]]
# val_cids = ["a"]
val_cids = [286, 6587, 6562, 11203]

train_names = [i for i in os.listdir("./images") if int(i.split("_")[0]) in train_cids]
val_names = [i for i in os.listdir("./images") if int(i.split("_")[0]) in val_cids]
from PIL import ImageOps
BATCH_SIZE = 16
def getitem(index, train=True):
  global val_names, train_names
  Y = []
  imgs = []

  # train=FalseTAMADE WEISM TRAIN = FALSE!!!guaibude bunengfanhua shouzhima ghuainbbude val namehao zuibakouke azhiqianweismkeyile? jiushiyingweimeiyouyongnamesle
  names = train_names if train else val_names
  start = index*BATCH_SIZE
  end =  (index+1)*BATCH_SIZE
  end = len(names) if end>len(names) else end
  # print(start, end)
  for i in range(start, end):


    name = random.choice([i for i in names])
    # name = random.choice([i for i in os.listdir("./images")]) #shetoudzikunhzhegegebuhaishisuijidemaguaibude

    img = Image.open(f"images/{name}").convert("RGB")
    if train:
      img = img.rotate(random.choice([0,90,180,270]), expand=0).resize((400,400))
      if random.random()>0.5:
        img = ImageOps.flip(img)
      if random.random()>0.5:
        img = ImageOps.mirror(img)
      img.crop((random.random()*30,random.random()*30,400-random.random()*30,400-random.random()*30))
    img = np.array(img.resize((400,400)), dtype="float32")
    if train:
      # img[:,:,0] *= random.random()*0.2+0.9
      img[:,:,1] *= random.random()*0.1+0.9
      img[:,:,2] *= random.random()*0.1+0.9
      img[:,:,:] *= random.random()*0.1+0.8
    img = torch.tensor(np.array(img)).permute(2, 0, 1).to(torch.float32).to(device)
    # if train:
    #   # img+=torch.rand((3,400,400)).to(device)*50
    #   for _ in range(30):
    #     x,y = int(random.random()*380),int(random.random()*380)
    #     img[:,x:x+20, y:y+20] = torch.rand((3,20,20))*250
#  guoranhaishiyaoqudizokunduzikoue
    # if train:
      # img*=torch.round(torch.tanh(5*torch.rand((3,400,400)))).to(device)
    # print(name)
    # print(converte//r.encode(molecules_rows[molecules_rows["cid"]==int(name.split("_")[0])]["canonicalsmiles"].item()))
    # length = (converter.encode(molecules_rows[molecules_rows["cid"]==int(name.split("_")[0])]["canonicalsmiles"].item()))
    length = molecules_rows[molecules_rows["cid"]==int(name.split("_")[0])]["canonicalsmiles"].item()
    Y.append(length)
    imgs.append(img)
  return torch.stack(imgs), Y


In [9]:
class Image2SMILES(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.decoder.encoder_dim = torch.nn.Dropout(0.2)

  def forward(self, image, text_in, xmask):
    image_feature = self.encoder(image)
    out = self.decoder(text_in, image_feature, x_mask=xmask)
    return out


model = torch.load("drive/MyDrive/encoder", map_location=device) #it is model not dict


In [10]:
class head(torch.nn.Module):
  def __init__(self, model):
    super().__init__()
    self.eff = model.eff
    self.pooling = torch.nn.AdaptiveAvgPool2d(output_size=1)
    self.projection = torch.nn.Linear(1280,16).to(device)
    self.projection3 = torch.nn.Linear(16,1).to(device)
  def forward(self, images):
    features = self.eff(images)
    features = torch.nn.functional.relu(self.projection(self.pooling(features).squeeze(-1).squeeze(-1)))
    features = torch.nn.functional.relu(torch.nn.functional.dropout(self.projection3(features), 0.2))
    # features = torch.nn.functional.relu(self.projection3(features))
    return features

In [11]:
mod = head(model)

In [12]:
def val():
  global val_names
  np.random.shuffle(val_names)
  running_loss = 0
  eval_mod = mod.train(False).to(device)

  image, Y = getitem(0, train=False)

  image = image.to(device)


  outputs = eval_mod(image)
  loss = torch.nn.functional.mse_loss(outputs, torch.tensor(Y).to(device).to(torch.float32).unsqueeze(-1))

  return loss.item()


In [13]:
def train(lr, gamma, epoch, forzen, reload=False, split=1, pretrain="mol_80k"):
  global mod
  # lr = lr*split #oh why learning rate*4 this is why it trains much better, still need nigher learning rate zuibagankouke zhege haizaizheli a a banbenwanquanluanle
  if reload:
    model = torch.load("drive/MyDrive/encoder", map_location=device) #it is model not dict
    mod = head(model)

  losses = []
  access = []
  val_acc = []
  val_loss = []
  optimizer = torch.optim.AdamW(
    mod.parameters(),
    lr=lr)
  import pylab
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)
  running_loss = 0
  for epoch in range(epoch):
    np.random.shuffle(train_names)
    mod = mod.train(True)

    for i in range(len(train_names)//(BATCH_SIZE*split)):
      if i==None:
        continue
      mod = mod.train(True)
      start_index = i
      image, Y = getitem(i, True)
      image = image.to(device)
      optimizer.zero_grad()
      outputs = mod(image)
      loss = torch.nn.functional.mse_loss(outputs, torch.tensor(Y).to(device).to(torch.float32).unsqueeze(-1))
      loss.backward()
      optimizer.step()
      losses.append(loss.item())
      if i%5==0:
        a = val()
        print(a)
        val_loss.append(a)
        scheduler.step()
  pylab.plot(losses, label="Train")
  pylab.plot(np.arange(1, len(val_loss)+1)*(len(losses)/len(val_loss)),val_loss,label="Val")
  pylab.legend()
  pylab.show()

In [14]:
train(0.0008, 0.985, 3,  True, reload=True, pretrain="mol_80k") #yeah need 0.001, if too smalel 0.001 cannot be trained duzikunduziex

7.081100940704346
2.212860584259033
4.207893371582031
10.90593147277832
3.3974390029907227
1.9236690998077393
2.6744189262390137
1.168203353881836
3.1036674976348877
2.8819310665130615
1.5791001319885254
1.9181008338928223
4.506965637207031
1.6834449768066406
3.6150169372558594
0.7927945852279663
3.6349658966064453
2.0566930770874023
3.8292250633239746
3.0924265384674072
2.009941339492798
2.035336971282959
3.4338865280151367
1.463343858718872
2.0759849548339844
3.689087152481079
1.7098761796951294
1.8024632930755615
0.4445168375968933


KeyboardInterrupt: 

In [ ]:
import gc
mod = mod.cpu()
model = model.cpu()
gc.collect()
torch.cuda.empty_cache()